In [6]:
import pandas as pd
df_Describe=pd.read_csv("/kaggle/input/datasets/dana3333/day-159/day15_real_dataset_large.csv")
print(df_Describe.head())
print(df_Describe.info())
print(df_Describe.nunique().sort_values(ascending=True).head(10))

cleaning_pipeline={
"age":{"type":"float","missing":"mean","outliers":"..."},
"income":{"type":"float","missing":"median","outliers":"..."},
"city":{"type":"categorical","clean":"canonical_mapping"},
"signup_time":{"type":"datetime","tz":"UTC"}}

df_start=df_Describe.copy()

def cleaning_step(df_start):
    df_start["income"]=pd.to_numeric(df_start["income"],errors="coerce")
    df_start["age"]=pd.to_numeric(df_start["age"],errors="coerce")
    df_start["signup_time"]=pd.to_datetime(df_start["signup_time"],errors="coerce")
    df_start["missing_per_row"]=df_start.isna().sum(axis=1)
    df_start["age"]=df_start["age"].fillna(df_start["age"].mean())
    df_start["income"]=df_start["income"].fillna(df_start["income"].median())
    Q1=df_start["income"].quantile(0.25)
    Q3=df_start["income"].quantile(0.75)
    IQR=Q3-Q1
    lower=Q1-1.5*IQR
    upper=Q3+1.5*IQR
    df_start["income"]=df_start["income"].clip(lower,upper)
    mean_age=df_start["age"].mean()
    std_age=df_start["age"].std()
    z_scores=(df_start["age"]-mean_age)/std_age
    df_start=df_start[abs(z_scores)<=3].copy()
    df_start["city_clean_basic"]=df_start["city"].str.strip().str.lower()
    df_start["city_clean_sep"]=(df_start["city_clean_basic"].str.replace("-"," ",regex=False)
    .str.replace(r"[^a-z\s]","",regex=True)
    .str.replace(r"\s+"," ",regex=True).str.strip())
    canonical_map={"new york":"new york","nyc":"new york","ny":"new york","san francisco":"san francisco","sanfrancisco":"san francisco"}
    df_start["city_token"]=df_start["city_clean_sep"].str.replace(" ","",regex=False)
    df_start["city_canonical"]=df_start["city_token"].map(canonical_map).fillna(df_start["city_clean_sep"])
    df_start["signup_time"]=df_start["signup_time"].dt.tz_localize("UTC")
    return df_start

df_clean=cleaning_step(df_start)
print(df_clean.head())
print(df_clean.info())
print(df_clean.nunique().sort_values(ascending=True).head(10))

cleaning_decisions={
"age":"Converted to numeric for calculations, filled missing values with the mean because age is roughly normally distributed, and removed extreme values using Z-score to keep ages realistic.",
"income":"Converted to numeric for analysis, filled missing values with the median since income is skewed and affected by high values, and capped outliers using IQR to reduce their impact without removing records.",
"city":"Cleaned and standardized city names to avoid duplicate categories with different spellings and make categorical analysis more accurate.",
"signup_time":"Converted to datetime and unified the timezone to UTC to ensure consistent and correct time-based comparisons."}
df_clean.to_csv("cleaned_dataset.csv",index=False)



       age    income     city signup_time
0       30   55000.0       NY  2024-01-01
1      NaN   70000.0       SF  2024-01-05
2       45       NaN       LA  not a date
3  unknown  120000.0       NY  2024/02/01
4       28   65000.0  Chicago  2024-01-15
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          78 non-null     object 
 1   income       77 non-null     float64
 2   city         96 non-null     object 
 3   signup_time  97 non-null     object 
dtypes: float64(1), object(3)
memory usage: 3.2+ KB
None
age            30
city           59
income         62
signup_time    95
dtype: int64
         age    income     city               signup_time  missing_per_row  \
0  30.000000   55000.0       NY 2024-01-01 00:00:00+00:00                0   
1  34.072464   70000.0       SF 2024-01-05 00:00:00+00:00                1   
2  45.000000   750